In [2]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1,998 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,026 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-23 02:05:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2023-03-23 02:05:48 (6.01 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...|2013-01-14 00:00:00|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...| 

### Create DataFrames to match tables

In [6]:
#from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
from pyspark.sql.functions import *
from pyspark.sql.functions import col


In [7]:

# Using agg count
# empDF.agg({'gender':'count','name':'count'}).show()

# from pyspark.sql.functions import *
# df=spark.createDataFrame([["02-03-2013"],["05-06-2023"]],["input"])
# df.select(col("input"),to_date(col("input"),"MM-dd-yyyy").alias("date")) \
#   .show()



In [8]:
# Create the vine_table. DataFrame
df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

### Analyze the VIne Data 

In [9]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [10]:
# Filter the Vine Data for Total Votes >= 20
from pyspark.sql.functions import *

from pyspark.sql.functions import col
#df = vine_df 

votes20p_df = df.filter(col("total_votes") >= 20)
votes20p_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|     

In [17]:
# Filter the Previous Dataset for records where the ratio of helpful_vote/tot_votes is >= 50%

from pyspark.sql.functions import col
votes50p_df = votes20p_df.filter(col("helpful_votes")/col("total_votes") >= .50)
votes50p_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|                Y|
|R230W87COFSTCL|          5|           21|         21|   N|     

In [18]:
#Filter the previous dataset and create a new dataframe with the rows where a review WAS written as part of the Vine programv
viney_df = votes50p_df.filter(col("vine")== "Y")
viney_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6U9701C3BGO6|          3|          139|        147|   Y|                N|
|R1XK3ALB45D7N4|          5|           33|         34|   Y|                N|
|R1IZCSTLX81D6C|          5|           31|         33|   Y|                N|
|R2C8NC8EQLH4JF|          3|           45|         48|   Y|                N|
|R1JJ1YOJMOML1P|          5|           18|         21|   Y|                N|
|R3TKG664L9MTXJ|          4|          164|        175|   Y|                N|
|R2E942L5EX73FP|          5|           29|         33|   Y|                N|
|R3CGRT9HYB3LI7|          5|           20|         21|   Y|                N|
|R3HIFO4D3ZIBZ7|          5|           25|         26|   Y|                N|
|R2VUXJT91MXOQJ|          3|           72|         79|   Y|     

In [19]:
#Filter the previous dataset and create a new dataframe with the rows where a review WAS NOT written as part of the Vine program
vinen_df = votes50p_df.filter(col("vine")== "N")
vinen_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|                Y|
|R230W87COFSTCL|          5|           21|         21|   N|     

In [20]:
#Count the total number of reviews.
from pyspark.sql.functions import count
votes50p_df.select(count(votes50p_df.review_id)).show()

+----------------+
|count(review_id)|
+----------------+
|           45421|
+----------------+



In [26]:
#Count the total number of reviews. Checked the original dataset df.groupBy("vine").count().show()
votes50p_df.groupBy("vine").count().show()


+----+-----+
|vine|count|
+----+-----+
|   Y|   33|
|   N|45388|
+----+-----+



In [22]:
viney_df.select(count(viney_df.review_id)).show()

+----------------+
|count(review_id)|
+----------------+
|              33|
+----------------+



In [29]:
#Summarize Vine Data
viney_df.groupby("star_rating").agg({'star_rating':'count','total_votes':'count'}).show()

+-----------+------------------+------------------+
|star_rating|count(total_votes)|count(star_rating)|
+-----------+------------------+------------------+
|          3|                10|                10|
|          5|                15|                15|
|          4|                 7|                 7|
|          2|                 1|                 1|
+-----------+------------------+------------------+



In [30]:
#Summarize Non - Vine Data
vinen_df.groupby("star_rating").agg({'star_rating':'count','total_votes':'count'}).show()

+-----------+------------------+------------------+
|star_rating|count(total_votes)|count(star_rating)|
+-----------+------------------+------------------+
|          1|              6379|              6379|
|          3|              4388|              4388|
|          5|             23733|             23733|
|          4|              7803|              7803|
|          2|              3085|              3085|
+-----------+------------------+------------------+

